<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/CNN/CNN_%3CUNK%3E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 1-import library
 
  إستيراد المكتبات

In [35]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import nltk
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

--------

2- import Datasets

استيراد ملفات البيانات


In [36]:
dataset = pd.read_excel("drive/MyDrive/mining/1.xlsx")
# dataset = pd.read_excel("drive/MyDrive/mining/2.xlsx")
# dataset = pd.read_excel("drive/MyDrive/mining/3.xlsx")
# ddatasetf = pd.read_excel("drive/MyDrive/mining/4.xlsx")

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17500 entries, 0 to 17499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    17361 non-null  object
 1   targe   17500 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 273.6+ KB


------
3- shuffle  
بعثرة البيانات

In [37]:
from sklearn.utils import shuffle
dataset=shuffle(dataset)
dataset

,text,targe
1411,علمت الصباح أن الفيلم الجديد للمخرج نبيل عيوش ...,0
7555,يستعد المغرب لإطلاق أول دفعة من السندات الإسلا...,2
4930,استقلال القاضي هدف ثابت وحتمي لتحقيق العدالة ف...,1
10054,بلغ حجم الإعفاءات الضريبية خلال العام الحالي م...,2
2714,أخبارنا المغربية سناء الوردي استغلت الفنانة ال...,0
...,...,...
11284,اختتمت شبيبة حزب التجمع الوطني للأحرار أشغال ج...,3
11964,عب رت الفعاليات الثقافية والفكرية الملتئمة ضمن...,3
5390,اعترضوه بالمدينة العتيقة ومارسوا عليه الجنس بع...,1
860,تستعد لتقديم برنامجين تلفزيونيين جديدين بعد رم...,0


------
4- clean
تنظيف البيانات

In [38]:
import string
import re
from nltk.stem.isri import ISRIStemmer

print(np.sum(dataset.isnull().any(axis=1)))
dataset= dataset.dropna()
print(np.sum(dataset.isnull().any(axis=1)))

#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
           
remove_hashtag(dataset)
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc
    
stemmer = ISRIStemmer()
dataset["text"] = dataset['text'].apply(lambda x: processDocument(x, stemmer))
print(dataset)

139
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


                                                    text  targe
1411   علمت الصباح ان الفيلم الجديد لمخرج نبيل عيوش س...      0
7555   يستعد المغرب لاطلاق اول دفعه من السندات الاسلا...      2
4930   استقلال القاضي هدف ثابت وحتمي لتحقيق العداله ف...      1
10054  بلغ حجم الاعفاءات الضريبيه خلال العام الحالي م...      2
2714   اخبارنا المغربيه سناء الوردي استغلت الفنانه ال...      0
...                                                  ...    ...
11284  اختمت شبيبه حزب التجمع الوطني لاحرار اشغال جام...      3
11964  عب رت الفعاليات الثقافيه والفكريه الملتئمه ضمن...      3
5390   اعترضوه بالمدينه العتيقه ومارسوا عليه الجنس بع...      1
860    تستعد لتقديم برنامجين تلفزيونين جديدين بعد رمض...      0
15795  اغلق امس الثلاثاء سوق الانتقالات الصيفيه بصفقا...      4

[17361 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [39]:
def Tokenizer_UNK(train_x,test_reviews,train_y, test_y):
  norm_train_reviews = (train_x)
  norm_test_reviews = (test_reviews)

  le = LabelEncoder()
  y_train = le.fit_transform(train_y)
  y_test = le.transform(test_y)

  from keras.utils.np_utils import to_categorical
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)
  # print(y[0])
  t = Tokenizer(oov_token='<UNK>')
  # fit the tokenizer on the documents
  t.fit_on_texts(norm_train_reviews)
  t.word_index['<PAD>'] = 0
  # max([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), min([(k, v) for k, v in t.word_index.items()], key = lambda x:x[1]), t.word_index['<UNK>']
  train_sequences = t.texts_to_sequences(norm_train_reviews)
  test_sequences = t.texts_to_sequences(norm_test_reviews)
  print("Vocabulary size={}".format(len(t.word_index)))
  print("Number of Documents={}".format(t.document_count))

  MAX_SEQUENCE_LENGTH = 1000
  # pad dataset to a maximum review length in words
  X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
  X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
  # X_train.shape, X_test.shape
  return X_train, X_test ,y_train,y_test

In [40]:
VOCAB_SIZE = len(t.word_index)
MAX_SEQUENCE_LENGTH = 1000
EMBED_SIZE = 300
EPOCHS=2
BATCH_SIZE=128
# create the model
model = Sequential()
model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=MAX_SEQUENCE_LENGTH))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))

# model.add(Dense(1, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))
# model.add(Flatten())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1000, 300)         48029100  
                                                                 
 conv1d_9 (Conv1D)           (None, 1000, 128)         153728    
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 500, 128)         0         
 1D)                                                             
                                                                 
 conv1d_10 (Conv1D)          (None, 500, 64)           32832     
                                                                 
 max_pooling1d_10 (MaxPoolin  (None, 250, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_11 (Conv1D)          (None, 250, 32)          

In [23]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [25]:
%%time
from sklearn.model_selection import StratifiedKFold
from keras.utils.np_utils import to_categorical
import os.path
from tensorflow import keras

# build train and test datasets
reviews = dataset['text'].values
sentiments = dataset['targe'].values
#####
strtfdKFold = StratifiedKFold(n_splits=4)#5
kfold = strtfdKFold.split(reviews, sentiments)
result_CNNClassifier=[]
#
for k, (train_index, test_index) in enumerate(kfold):
    # if(os.path.exists("drive/MyDrive/mining/CNN.net")):
    #   print("load net")
    #   model = keras.models.load_model('drive/MyDrive/mining/CNN.net') 

    X_train_, X_test_ = reviews[train_index], reviews[test_index]
    y_train_, y_test_ = sentiments[train_index], sentiments[test_index]

    X_train, X_test ,y_train,y_test=Tokenizer_UNK(X_train_,X_test_,y_train_,y_test_)

    # Fit the model
    model.fit(X_train, y_train, validation_split=0.1, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)
    y_pred=(model.predict(X_test))
    y_classes_pre = [np.argmax(y, axis=None, out=None) for y in y_pred]
    y_classes_test = [np.argmax(y, axis=None, out=None) for y in y_test]
    result_CNNClassifier.append(calculate_results(y_classes_test, y_classes_pre))
    # model.save('drive/MyDrive/mining/CNN.net')
    # print("save net")

result_CNNClassifier

Vocabulary size=154495
Number of Documents=13020
Epoch 1/2
92/92 [==============================] - 10s 99ms/step - loss: 0.3257 - accuracy: 0.5790 - val_loss: 0.0948 - val_accuracy: 0.9201
Epoch 2/2
92/92 [==============================] - 8s 90ms/step - loss: 0.0479 - accuracy: 0.9643 - val_loss: 0.0565 - val_accuracy: 0.9562
Vocabulary size=155322
Number of Documents=13021
Epoch 1/2
92/92 [==============================] - 9s 94ms/step - loss: 0.1334 - accuracy: 0.8746 - val_loss: 0.0542 - val_accuracy: 0.9524
Epoch 2/2
92/92 [==============================] - 8s 92ms/step - loss: 0.0114 - accuracy: 0.9933 - val_loss: 0.0526 - val_accuracy: 0.9555
Vocabulary size=155006
Number of Documents=13021
Epoch 1/2
92/92 [==============================] - 9s 93ms/step - loss: 0.1211 - accuracy: 0.8862 - val_loss: 0.0501 - val_accuracy: 0.9563
Epoch 2/2
92/92 [==============================] - 8s 92ms/step - loss: 0.0085 - accuracy: 0.9935 - val_loss: 0.0569 - val_accuracy: 0.9578
Vocabulary s

In [28]:
accuracy=0
f1=0
precision=0
recall=0
for i in result_CNNClassifier:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(result_CNNClassifier)
f1=f1/len(result_CNNClassifier)
precision=precision/len(result_CNNClassifier)
recall=recall/len(result_CNNClassifier)
# (result_DecisionTreeClassifier)
results_CNNClassifier=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]
results_CNNClassifier

[{'accuracy': 96.1638107658517,
  'f1': 0.9616706026038523,
  'precision': 0.9623011018202574,
  'recall': 0.961638107658517}]

In [29]:
print('DecisionTreeClassifier results in dataset 1:')
results_CNNClassifier

DecisionTreeClassifier results in dataset 1:


[{'accuracy': 96.1638107658517,
  'f1': 0.9616706026038523,
  'precision': 0.9623011018202574,
  'recall': 0.961638107658517}]

In [ ]:
print('DecisionTreeClassifier results in dataset 2:')
results_CNNClassifier

In [ ]:
print('DecisionTreeClassifier results in dataset 3:')
results_CNNClassifier

In [ ]:
print('DecisionTreeClassifier results in dataset 4:')
results_CNNClassifier

In [ ]:
Final_Result=[
 
  ,
  
  ,

  ,
 
  ]

accuracy=0
f1=0
precision=0
recall=0
for i in Final_Result:
  accuracy+=(i['accuracy'])
  f1+=(i['f1'])
  precision+=(i['precision'])
  recall+=(i['recall'])

accuracy=accuracy/len(Final_Result)
f1=f1/len(Final_Result)
precision=precision/len(Final_Result)
recall=recall/len(Final_Result)
# (result_DecisionTreeClassifier)
Final_Result=[{'accuracy':accuracy,'f1':f1,'precision':precision,'recall':recall}]

print('Final_Result:')
print(Final_Result)

---------------
